In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName('KafkaSparkStreaming') \
    .master("spark://spark-master:7077") \
    .config("spark.executor.cores", "2") \
    .config("spark.executor.memory", "2g") \
    .config("spark.cores.max", "4") \
    .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.1') \
    .config("spark.hadoop.fs.defaultFS", "hdfs://hdfs-namenode:8020") \
    .getOrCreate()

df = spark \
  .readStream \
  .format('kafka') \
  .option('kafka.bootstrap.servers', 'kafka:9092') \
  .option('subscribe', 'test-topic') \
  .load()

In [ ]:
#write to HDFS isntead of console
df.selectExpr('CAST(key AS STRING)', 'CAST(value AS STRING)') \
  .writeStream \
  .format('csv') \
  .option('checkpointLocation', '/tmp/checkpoint') \
  .option('path', '/tmp/data') \
  .start() \
  .awaitTermination()

In [ ]:
spark.stop()

# Check Loaded data

In [ ]:
df = spark.read.csv("/tmp/data/*", header=False, inferSchema=True)

In [ ]:
df.show()